In [ ]:
import os
import torch
from torch import nn
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes, knn_points, knn_gather
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
import numpy as np
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import plotly.express as px

import sys
sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src')
from shapeaxi import utils

from shapeaxi.saxi_nets import MHAIdxEncoder, MHAIdxDecoder
import pandas as pd

In [ ]:
device = torch.device("cuda:0")


In [ ]:
# target_fn = '/mnt/famli_netapp_shared/C1_ML_Analysis/src/diffusion-models/blender/studies/placenta/FAM-025-0499-5/brain/leftWhiteMatter.stl'
# target = utils.ReadSurf(target_fn)
# target, target_mean_bb, target_scale_factor = utils.ScaleSurf(target)
# target_v, target_f, target_e = utils.PolyDataToTensors(target, device=device)
# target_mesh = Meshes(verts=[target_v], faces=[target_f])

In [ ]:
def plot_pointcloud(mesh, title=""):
    points = sample_points_from_meshes(mesh, 5000)
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)    
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z)])
    fig.show()

In [ ]:

encoder = MHAIdxEncoder(input_dim=1, output_dim=64, K=[None, None, (96, 32), (96, 32)], num_heads=[None, None, 64, 128], stages=[16, 32, 64, 128], conv_v_kernel_size=[5, 5, 7, 7], dropout=0.1, pooling_factor=[0.125, 0.125, 0.125, 0.125], pooling_hidden_dim=[8, 16, 32, 64], pooling_K=[27, 27, 27, 27], feed_forward_hidden_dim=[8, 16, 32, 64], score_pooling=False, time_embed_dim=128)

encoder.to(device)




In [ ]:
BS = 6
N = 250000
X = torch.rand(BS, N, 1).to(device)
X_v = torch.rand(BS, N, 3).to(device)
# print(x.shape)
# x_v_fixed = torch.rand(BS, 10000, 3).to(device)

x, unpooling_idxs  = encoder(X, X_v)

In [ ]:
x.shape

In [ ]:

decoder = MHAIdxDecoder(input_dim=64, output_dim=1, K=[(96, 32), (96, 32), None, None], num_heads=[64, 32, None, None], stages=[128, 64, 32, 16], dropout=0.1, pooling_hidden_dim=[64, 32, 16, 8], conv_v_kernel_size=[5, 5, 5, 5], feed_forward_hidden_dim=[64, 32, 16, 8])
decoder.to(device)




In [ ]:
X_hat = decoder(x, unpooling_idxs)